In [11]:
import time #To check execution time

import numpy as np
from numpy.polynomial import polynomial as poly

def polymul(x, y, modulus, poly_mod):
    """Add two polynoms
    Args:
        x, y: two polynoms to be added.
        modulus: coefficient modulus.
        poly_mod: polynomial modulus.
    Returns:
        A polynomial in Z_modulus[X]/(poly_mod).
    """
    return np.int64(
        np.round(poly.polydiv(poly.polymul(x, y) % modulus, poly_mod)[1] % modulus)
    )


def polyadd(x, y, modulus, poly_mod):
    """Multiply two polynoms
    Args:
        x, y: two polynoms to be multiplied.
        modulus: coefficient modulus.
        poly_mod: polynomial modulus.
    Returns:
        A polynomial in Z_modulus[X]/(poly_mod).
    """
    return np.int64(
        np.round(poly.polydiv(poly.polyadd(x, y) % modulus, poly_mod)[1] % modulus)
    )

## Binary Polynomial 

The binary coefficients (0 or 1) define the polynomial. In this example, the polynomial is 

\[ P(x) = x^4 + x^2 + x + 1 \]

which means:

\[ 
P(x) = 1 \cdot x^4 + 0 \cdot x^3 + 1 \cdot x^2 + 1 \cdot x + 1 
\]

## Uniform Polynomial

The coefficients are uniformly sampled integers between 0 and the specified modulus (7 in this case). Here, the polynomial is:

\[ 
P(x) = 6x^4 + 5x^3 + 3x^2 + 4x + 2 
\]

## Normal Polynomial

The coefficients are sampled from a normal distribution and then rounded to integers. The result might include negative coefficients:

\[ 
P(x) = 3x^4 + 2x^3 - x + 1 
\]


In [12]:
def gen_binary_poly(size):
    return np.random.randint(0, 2, size, dtype=np.int64)

def gen_uniform_poly(size, modulus):
    return np.random.randint(0, modulus, size, dtype=np.int64)

def gen_normal_poly(size):
    return np.int64(np.random.normal(0, 2, size=size)) #Used to generate random numbers from a normal (or Gaussian) distribution.


def poly_to_string(coeffs):
    """
    Converts a polynomial (in coefficient form) to its string representation.
    Args:
        coeffs: List or array of polynomial coefficients, where coeff[i]
                is the coefficient of x^i.
    Returns:
        String representation of the polynomial.
    """
    terms = []
    degree = len(coeffs) - 1
    for i, coeff in enumerate(coeffs):
        if coeff == 0:
            continue
        elif i == 0:
            terms.append(f"{coeff}")
        elif i == 1:
            terms.append(f"{coeff}x")
        else:
            terms.append(f"{coeff}x^{i}")
    return " + ".join(terms[::-1]) if terms else "0"

# Example usage:
size = 5
modulus = 7  # For the uniform polynomial
binary_poly = gen_binary_poly(size)
uniform_poly = gen_uniform_poly(size, modulus)
normal_poly = gen_normal_poly(size)

print("Binary Polynomial:", poly_to_string(binary_poly))
print("Uniform Polynomial:", poly_to_string(uniform_poly))
print("Normal Polynomial:", poly_to_string(normal_poly))

Binary Polynomial: 1x^4 + 1x^3 + 1x^2 + 1
Uniform Polynomial: 6x^4 + 4x^3 + 1x^2 + 4x + 5
Normal Polynomial: -1x^4 + -2x + 2



- Binary polynomials (like \(sk\)) are used because of their simplicity and security properties.
- Uniform polynomials (like \(a\)) add randomness to the public key, making the encryption process secure.
- Error polynomials (like \(e\)) introduce noise to the encryption, ensuring that even if part of the public key is known, the underlying secret key remains difficult to determine.
- The operations on the polynomials (addition and multiplication under moduli) ensure that the cryptographic system operates over a finite ring, which is crucial for security.


# Key Generation

The function returns the public key \((b, a)\) and the secret key \(sk\).

### Public Key

The public key is represented as:\[(b, a)\]

- Both polynomials \(b\) and \(a\) are used in the encryption process.
- Where, $b = (a \;. \;sk\; +\; e) (mod \; poly\_mod)$

### Secret Key

The secret key is represented as:\[sk\]

- This key is used for decryption.

In [13]:
def keygen(size, modulus, poly_mod):
    """Generate a public and secret keys
    Args:
        size: size of the polynoms for the public and secret keys.
        modulus: coefficient modulus.
        poly_mod: polynomial modulus.
    Returns:
        Public and secret key.
    """
    sk = gen_binary_poly(size)  #Secret Key
    a = gen_uniform_poly(size, modulus) #
    e = gen_normal_poly(size)
    b = polyadd(polymul(-a, sk, modulus, poly_mod), -e, modulus, poly_mod)
    return (b, a), sk

In [14]:
def encrypt(pk, size, q, t, poly_mod, pt):
    """Encrypt an integer.
    Args:
        pk: public-key.
        size: size of polynomials.
        q: ciphertext modulus.
        t: plaintext modulus.
        poly_mod: polynomial modulus.
        pt: integer to be encrypted.
    Returns:
        Tuple representing a ciphertext.      
    """
    # encode the integer into a plaintext polynomial
    m = np.array([pt] + [0] * (size - 1), dtype=np.int64) % t
    delta = q // t
    scaled_m = delta * m  % q
    e1 = gen_normal_poly(size)
    e2 = gen_normal_poly(size)
    u = gen_binary_poly(size)
    ct0 = polyadd(
            polyadd(
                polymul(pk[0], u, q, poly_mod),
                e1, q, poly_mod),
            scaled_m, q, poly_mod
        )
    ct1 = polyadd(
            polymul(pk[1], u, q, poly_mod),
            e2, q, poly_mod
        )
    return (ct0, ct1)

In [15]:
def decrypt(sk, size, q, t, poly_mod, ct):
    """Decrypt a ciphertext
    Args:
        sk: secret-key.
        size: size of polynomials.
        q: ciphertext modulus.
        t: plaintext modulus.
        poly_mod: polynomial modulus.
        ct: ciphertext.
    Returns:
        Integer representing the plaintext.
    """
    scaled_pt = polyadd(
            polymul(ct[1], sk, q, poly_mod),
            ct[0], q, poly_mod
        )
    decrypted_poly = np.round(scaled_pt * t / q) % t
    return int(decrypted_poly[0])

In [16]:
def add_plain(ct, pt, q, t, poly_mod):
    """Add a ciphertext and a plaintext.
    Args:
        ct: ciphertext.
        pt: integer to add.
        q: ciphertext modulus.
        t: plaintext modulus.
        poly_mod: polynomial modulus.
    Returns:
        Tuple representing a ciphertext.
    """
    size = len(poly_mod) - 1
    # encode the integer into a plaintext polynomial
    m = np.array([pt] + [0] * (size - 1), dtype=np.int64) % t
    delta = q // t
    scaled_m = delta * m  % q
    new_ct0 = polyadd(ct[0], scaled_m, q, poly_mod)
    return (new_ct0, ct[1])

In [17]:
def mul_plain(ct, pt, q, t, poly_mod):
    """Multiply a ciphertext and a plaintext.
    Args:
        ct: ciphertext.
        pt: integer to multiply.
        q: ciphertext modulus.
        t: plaintext modulus.
        poly_mod: polynomial modulus.
    Returns:
        Tuple representing a ciphertext.
    """
    size = len(poly_mod) - 1
    # encode the integer into a plaintext polynomial
    m = np.array([pt] + [0] * (size - 1), dtype=np.int64) % t
    new_c0 = polymul(ct[0], m, q, poly_mod)
    new_c1 = polymul(ct[1], m, q, poly_mod)
    return (new_c0, new_c1)

In [23]:
# Scheme's parameters
# polynomial modulus degree
n = 2**4
# ciphertext modulus
q = 2**15
# plaintext modulus
t = 2**8
# polynomial modulus
poly_mod = np.array([1] + [0] * (n - 1) + [1])
# Keygen
pk, sk = keygen(n, q, poly_mod)
# Encryption
pt1, pt2 , pt3 = 73, 20, 10
cst1, cst2, cst3 = 7, 5, 10
ct1 = encrypt(pk, n, q, t, poly_mod, pt1)
ct2 = encrypt(pk, n, q, t, poly_mod, pt2)
ct3 = encrypt(pk, n, q, t, poly_mod, pt3)

print("[+] Ciphertext ct1({}):".format(pt1))
print("\n")
print("\t ct1_0:", ct1[0])
print("\t ct1_1:", ct1[1])
print("\n")
print("[+] Ciphertext ct2({}):".format(pt2))
print("\n")
print("\t ct1_0:", ct2[0])
print("\t ct1_1:", ct2[1])
print("\n")
print("[+] Ciphertext ct1({}):".format(pt3))
print("\n")
print("\t ct1_0:", ct3[0])
print("\t ct1_1:", ct3[1])
print("\n")

# Evaluation
ct4 = add_plain(ct1, cst1, q, t, poly_mod)
ct5 = mul_plain(ct2, cst2, q, t, poly_mod)
ct6 = mul_plain(ct3, cst3, q, t, poly_mod)

# Decryption
d_start_ct4_time = time.time() 
decrypted_ct4 = decrypt(sk, n, q, t, poly_mod, ct4)
d_end_ct4_time = time.time()
print("[+] Decrypted ct4(ct1 + {}): {}".format(cst1, decrypted_ct4))
print(f"Decrypt time::{(d_end_ct4_time - d_start_ct4_time)*1000} mili seconds \n")

d_start_ct4_time = time.time() 
decrypted_ct5 = decrypt(sk, n, q, t, poly_mod, ct5)
d_end_ct4_time = time.time()
print("[+] Decrypted ct5(ct2 * {}): {}".format(cst2, decrypted_ct5))
print(f"Decrypt time::{(d_end_ct4_time - d_start_ct4_time)*1000} mili seconds \n")

d_start_ct4_time = time.time() 
decrypted_ct6 = decrypt(sk, n, q, t, poly_mod, ct6)
d_end_ct4_time = time.time()
print("[+] Decrypted ct6(ct3 * {}): {}".format(cst3, decrypted_ct6))
print(f"Decrypt time::{(d_end_ct4_time - d_start_ct4_time)*1000} mili seconds \n")





[+] Ciphertext ct1(73):


	 ct1_0: [19948 17955  6398  1843 23189 22825 18249 28216 25573 22406  2125 19262
  9723 11304 25256 19313]
	 ct1_1: [15747 20314 14023  4064 14446  9771 29935  9122 28381  6121  8036  3204
  4849 16241  5167 25189]


[+] Ciphertext ct2(20):


	 ct1_0: [22659  3694 28116 19753 15308  9179 15468  9730 21023 12360   413  8205
 15902 30067 12518  3460]
	 ct1_1: [10522 23801 21479  8237  2781  1898 16367  3551 28229  8342 15012 27507
 12660 16688 12248 14311]


[+] Ciphertext ct1(10):


	 ct1_0: [31884 28700 30067  5374 14892  5765  3979  9959  7695 16996 15132  6420
  8990 24910 24451  8442]
	 ct1_1: [27384  1051 13348  2609 23817  9364 31044 15626 21803 29330  7585 16697
 29378  6107 30752 23631]


[+] Decrypted ct4(ct1 + 7): 80
Decrypt time::4.055023193359375 mili seconds 

[+] Decrypted ct5(ct2 * 5): 99
Decrypt time::3.925800323486328 mili seconds 

[+] Decrypted ct6(ct3 * 10): 99
Decrypt time::4.015207290649414 mili seconds 

